<a href="https://colab.research.google.com/github/thadduslee/NUS-Fintech-Summit-Blockchain/blob/main/NUS_fintech_summit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install xrpl-py

In [2]:
pip install nest_asyncio

In [3]:
import requests

def get_xrp_price_coingecko():
    """Fetches XRP price from CoinGecko (Works in Colab)"""
    url = "https://api.coingecko.com/api/v3/simple/price?ids=ripple&vs_currencies=usd"

    try:
        # CoinGecko requires a User-Agent to look like a real browser
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        data = response.json()

        # Extract price
        price = data['ripple']['usd']
        return float(price)

    except Exception as e:
        print(f"CoinGecko Failed: {e}")
        return None

# --- TEST IT ---
latest_xrp_price = get_xrp_price_coingecko()
print(f"Current XRP Price: ${latest_xrp_price}")

Current XRP Price: $2.17


# Simulate first semester of token issuance and initial sale

In [4]:
import nest_asyncio
nest_asyncio.apply()

import math
import json
from decimal import Decimal
from xrpl.clients import JsonRpcClient
from xrpl.wallet import generate_faucet_wallet
from xrpl.models.transactions import TrustSet, Payment, OfferCreate
from xrpl.models.amounts import IssuedCurrencyAmount
from xrpl.transaction import submit_and_wait
from xrpl.utils import xrp_to_drops

# --- 1. SETUP: Create 3 Characters ---
print("--- 1. GENERATING WALLETS ---")
client = JsonRpcClient("https://s.altnet.rippletest.net:51234/")

print("Creating Issuer (The Bank)...")
issuer = generate_faucet_wallet(client)
print("Creating Seller (The Merchant)...")
seller = generate_faucet_wallet(client)
print("Creating Buyer (The Customer)...")
buyer = generate_faucet_wallet(client)

school_fees = int(input("How much money do you need to cover the first semester: "))
xrps_needed = math.ceil(school_fees / latest_xrp_price)
price_per_token = xrps_needed / 125

CURRENCY_CODE = "PYT"
TOKEN_SUPPLY = "125"
SALE_AMOUNT = "1"
PRICE_IN_XRP = str(price_per_token)

# --- 2. ISSUANCE: Bank -> Seller ---
print(f"\n--- 2. ISSUING TOKENS ---")
# A. Seller trusts Bank
trust_tx = TrustSet(
    account=seller.classic_address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=issuer.classic_address,
        value="1000000000"
    )
)
submit_and_wait(trust_tx, client, seller)

# B. Bank sends tokens to Seller
payment_tx = Payment(
    account=issuer.classic_address,
    destination=seller.classic_address,
    amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=issuer.classic_address,
        value=TOKEN_SUPPLY
    )
)
submit_and_wait(payment_tx, client, issuer)
print("Tokens Issued to Seller.")

# --- 3. SELLER CREATES MARKET (The "Sell" Order) ---
print(f"\n--- 3. SELLER POSTS ORDER ---")
print(f"Seller: 'I am selling {SALE_AMOUNT} PYT for {PRICE_IN_XRP} XRP'")

sell_offer = OfferCreate(
    account=seller.classic_address,
    taker_gets={
        "currency": CURRENCY_CODE,
        "issuer": issuer.classic_address,
        "value": SALE_AMOUNT
    },
    taker_pays=xrp_to_drops(Decimal(PRICE_IN_XRP))
)
submit_and_wait(sell_offer, client, seller)
print("Sell Order is live on the DEX.")

# --- 4. BUYER ENTERS THE MARKET ---
print(f"\n--- 4. BUYER PURCHASES TOKENS ---")

# Important: Buyer MUST trust the Issuer before they can hold the token
buyer_trust = TrustSet(
    account=buyer.classic_address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=issuer.classic_address,
        value="1000000000"
    )
)
submit_and_wait(buyer_trust, client, buyer)

# Buyer places a matching offer
# Note: For Buyer, 'TakerGets' is XRP, and 'TakerPays' is PYT
print(f"Buyer: 'I am buying {SALE_AMOUNT} PYT for {PRICE_IN_XRP} XRP'")

buy_offer = OfferCreate(
    account=buyer.classic_address,
    taker_gets=xrp_to_drops(Decimal(PRICE_IN_XRP)), # Buyer offers XRP
    taker_pays={
        "currency": CURRENCY_CODE,
        "issuer": issuer.classic_address,
        "value": SALE_AMOUNT # Buyer wants PYT
    }
)
buy_response = submit_and_wait(buy_offer, client, buyer)

# --- 5. RESULTS ---
print(f"\n--- 5. FINAL REPORT ---")
# Check if the trade happened by looking at the transaction metadata
result_code = buy_response.result.get("meta", {}).get("TransactionResult")
print(f"Trade Status: {result_code}")

print("\nVerify on Explorer (Look for 'Trade' or 'Exchange'):")
print(f"https://testnet.xrpl.org/accounts/{buyer.classic_address}")

--- 1. GENERATING WALLETS ---
Creating Issuer (The Bank)...
Creating Seller (The Merchant)...
Creating Buyer (The Customer)...
How much money do you need to cover the first semester: 2780

--- 2. ISSUING TOKENS ---
Tokens Issued to Seller.

--- 3. SELLER POSTS ORDER ---
Seller: 'I am selling 1 PYT for 10.256 XRP'
Sell Order is live on the DEX.

--- 4. BUYER PURCHASES TOKENS ---
Buyer: 'I am buying 1 PYT for 10.256 XRP'

--- 5. FINAL REPORT ---
Trade Status: tesSUCCESS

Verify on Explorer (Look for 'Trade' or 'Exchange'):
https://testnet.xrpl.org/accounts/rDx6Mt224DRMCKy2VQ9wTJSzt9rByZLp3o


# Simulate activity on the secondary market for the student's tokens

In [5]:
temp = input("How many tokens would you like to sell: ")
temp2 = input("How much would you like to get per token: ")
temp2 = float(temp2) * float(temp)
SALE_AMOUNT = str(temp)
PRICE_IN_XRP = str(temp2)

# --- 3. SELLER CREATES MARKET (The "Sell" Order) ---
print(f"\n--- 3. SELLER POSTS ORDER ---")
print(f"Seller: 'I am selling {SALE_AMOUNT} PYT for {PRICE_IN_XRP} XRP'")

sell_offer = OfferCreate(
    account=seller.classic_address,
    taker_gets={
        "currency": CURRENCY_CODE,
        "issuer": issuer.classic_address,
        "value": SALE_AMOUNT
    },
    taker_pays=xrp_to_drops(Decimal(PRICE_IN_XRP))
)
submit_and_wait(sell_offer, client, seller)
print("Sell Order is live on the DEX.")

# --- 4. BUYER ENTERS THE MARKET ---
print(f"\n--- 4. BUYER PURCHASES TOKENS ---")

# Important: Buyer MUST trust the Issuer before they can hold the token
buyer_trust = TrustSet(
    account=buyer.classic_address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=issuer.classic_address,
        value="1000000000"
    )
)
submit_and_wait(buyer_trust, client, buyer)

# Buyer places a matching offer
# Note: For Buyer, 'TakerGets' is XRP, and 'TakerPays' is PYT
print(f"Buyer: 'I am buying {SALE_AMOUNT} PYT for {PRICE_IN_XRP} XRP'")

buy_offer = OfferCreate(
    account=buyer.classic_address,
    taker_gets=xrp_to_drops(Decimal(PRICE_IN_XRP)), # Buyer offers XRP
    taker_pays={
        "currency": CURRENCY_CODE,
        "issuer": issuer.classic_address,
        "value": SALE_AMOUNT # Buyer wants PYT
    }
)
buy_response = submit_and_wait(buy_offer, client, buyer)

# --- 5. RESULTS ---
print(f"\n--- 5. FINAL REPORT ---")
# Check if the trade happened by looking at the transaction metadata
result_code = buy_response.result.get("meta", {}).get("TransactionResult")
print(f"Trade Status: {result_code}")

print("\nVerify on Explorer (Look for 'Trade' or 'Exchange'):")
print(f"https://testnet.xrpl.org/accounts/{buyer.classic_address}")

How many tokens would you like to sell: 2
How much would you like to get per token: 20

--- 3. SELLER POSTS ORDER ---
Seller: 'I am selling 2 PYT for 40.0 XRP'
Sell Order is live on the DEX.

--- 4. BUYER PURCHASES TOKENS ---
Buyer: 'I am buying 2 PYT for 40.0 XRP'

--- 5. FINAL REPORT ---
Trade Status: tesSUCCESS

Verify on Explorer (Look for 'Trade' or 'Exchange'):
https://testnet.xrpl.org/accounts/rDx6Mt224DRMCKy2VQ9wTJSzt9rByZLp3o


In [19]:
# --- INPUTS ---
temp = input("How many tokens would you like to buy: ")
temp2 = input("How much would you like to pay per token: ")

# Calculate total XRP needed
temp2 = float(temp2) * float(temp)
BUY_AMOUNT = str(temp)
PRICE_IN_XRP = str(temp2)

# --- PRE-REQUISITE: TRUST LINE ---
print(f"\n--- PRE-REQ: BUYER SETS TRUST ---")
# Important: Buyer MUST trust the Issuer before they can post a Buy offer
buyer_trust = TrustSet(
    account=buyer.classic_address,
    limit_amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=issuer.classic_address,
        value="1000000000"
    )
)
submit_and_wait(buyer_trust, client, buyer)
print("Trust line established.")

# --- 3. BUYER CREATES MARKET (The "Buy" Order) ---
print(f"\n--- 3. BUYER POSTS ORDER ---")
print(f"Buyer: 'I am buying {BUY_AMOUNT} PYT for {PRICE_IN_XRP} XRP'")

# Buyer creates the offer: Giving XRP (TakerGets), Wanting Tokens (TakerPays)
buy_offer = OfferCreate(
    account=buyer.classic_address,
    taker_gets=xrp_to_drops(Decimal(PRICE_IN_XRP)), # Buyer offers XRP
    taker_pays={
        "currency": CURRENCY_CODE,
        "issuer": issuer.classic_address,
        "value": BUY_AMOUNT # Buyer wants PYT
    }
)
submit_and_wait(buy_offer, client, buyer)
print("Buy Order is live on the DEX.")

# --- 4. SELLER ENTERS THE MARKET ---
print(f"\n--- 4. SELLER SELLS TOKENS ---")
print(f"Seller: 'I am selling {BUY_AMOUNT} PYT to fill the buy order'")

# Seller places a matching offer: Giving Tokens (TakerGets), Wanting XRP (TakerPays)
sell_offer = OfferCreate(
    account=seller.classic_address,
    taker_gets={
        "currency": CURRENCY_CODE,
        "issuer": issuer.classic_address,
        "value": BUY_AMOUNT
    },
    taker_pays=xrp_to_drops(Decimal(PRICE_IN_XRP))
)
sell_response = submit_and_wait(sell_offer, client, seller)

# --- 5. RESULTS ---
print(f"\n--- 5. FINAL REPORT ---")
# Check if the trade happened by looking at the transaction metadata
result_code = sell_response.result.get("meta", {}).get("TransactionResult")
print(f"Trade Status: {result_code}")

print("\nVerify on Explorer (Look for 'Trade' or 'Exchange'):")
print(f"https://testnet.xrpl.org/accounts/{seller.classic_address}")

How many tokens would you like to buy: 5
How much would you like to pay per token: 12

--- PRE-REQ: BUYER SETS TRUST ---
Trust line established.

--- 3. BUYER POSTS ORDER ---
Buyer: 'I am buying 5 PYT for 60.0 XRP'
Buy Order is live on the DEX.

--- 4. SELLER SELLS TOKENS ---
Seller: 'I am selling 5 PYT to fill the buy order'

--- 5. FINAL REPORT ---
Trade Status: tesSUCCESS

Verify on Explorer (Look for 'Trade' or 'Exchange'):
https://testnet.xrpl.org/accounts/rQN2DFJP6esSWRtzhZHBKe1DYqp84Nr95h


# Find out how much the latest token was transacted for

In [6]:
from decimal import Decimal

# 1. SETUP: Get the data
# We assume 'buy_response' is the variable from your previous step
meta = buy_response.result.get("meta", {})
affected_nodes = meta.get("AffectedNodes", [])
buyer_address = buyer.classic_address

print(f"--- ANALYZING TRADE FOR {buyer_address} ---")

# 2. DETECT THE FEE (The fix for your error)
# We look in a few places. If we can't find it, we assume standard 12 drops.
fee_drops = buy_response.result.get("Fee")
if not fee_drops:
    # Try looking inside a nested 'transaction' object
    fee_drops = buy_response.result.get("transaction", {}).get("Fee")

if fee_drops:
    fee_decimal = Decimal(fee_drops)
else:
    print("Warning: Could not read Fee. Assuming standard 12 drops.")
    fee_decimal = Decimal("12")

print(f"Transaction Fee Paid: {fee_decimal} drops")

# 3. CALCULATE BALANCES
xrp_spent = Decimal(0)
tokens_received = Decimal(0)

found_changes = False

for node in affected_nodes:
    if "ModifiedNode" in node:
        entry = node["ModifiedNode"]

        # A. Check XRP Changes (AccountRoot)
        if entry["LedgerEntryType"] == "AccountRoot":
            if entry["FinalFields"]["Account"] == buyer_address:
                found_changes = True
                # Math: Old Balance - New Balance = Total Spent
                prev_xrp = Decimal(entry["PreviousFields"]["Balance"])
                new_xrp = Decimal(entry["FinalFields"]["Balance"])

                # Total Diff = (Trade Cost) + (Gas Fee)
                total_diff = prev_xrp - new_xrp

                # We remove the Gas Fee to find the actual Trade Cost
                trade_cost_drops = total_diff - fee_decimal
                xrp_spent = trade_cost_drops / Decimal("1000000") # Convert to XRP

        # B. Check Token Changes (RippleState)
        if entry["LedgerEntryType"] == "RippleState":
            # Check if this trust line involves our Currency (PYT)
            if entry["FinalFields"]["Balance"]["currency"] == CURRENCY_CODE:
                found_changes = True
                prev_token = Decimal(entry["PreviousFields"]["Balance"]["value"])
                new_token = Decimal(entry["FinalFields"]["Balance"]["value"])

                # Use abs() because the sign depends on who is high/low issuer
                tokens_received = abs(new_token - prev_token)

# 4. FINAL REPORT
if found_changes and tokens_received > 0:
    print("\n✅ SUCCESS: Trade Detected")
    print(f"Tokens Received: {tokens_received} {CURRENCY_CODE}")
    print(f"XRP Paid:        {xrp_spent} XRP")

    price = xrp_spent / tokens_received
    print(f"Price per Token: {price} XRP")
elif found_changes:
    print("\n⚠ Balance changed, but no tokens received.")
    print(f"XRP Spent: {xrp_spent} (Likely just gas fees for a failed offer)")
else:
    print("\n❌ No balance changes found.")
    print("This implies the transaction didn't execute or the variables are mismatched.")

--- ANALYZING TRADE FOR rDx6Mt224DRMCKy2VQ9wTJSzt9rByZLp3o ---
Transaction Fee Paid: 12 drops

✅ SUCCESS: Trade Detected
Tokens Received: 2 PYT
XRP Paid:        39.999998 XRP
Price per Token: 19.999999 XRP


# Use latest price of token sale to determine how many contracts can be minted for the next semester

In [18]:
from xrpl.models.transactions import Payment
from xrpl.models.amounts import IssuedCurrencyAmount
from xrpl.transaction import submit_and_wait
from math import floor
from decimal import Decimal # Import Decimal for precise calculations

# SETTINGS
school_fees = input("How much are the school fee's for the upcoming semester in USD: ")
number_of_xrp = int(school_fees) / latest_xrp_price

# Convert number_of_xrp to Decimal before dividing by price (which is also a Decimal)
temp = floor(Decimal(number_of_xrp) / price)
MORE_TOKENS = str(temp)

print(f"--- ISSUING {MORE_TOKENS} MORE {CURRENCY_CODE} ---")

# Define the Payment (Issuer -> Seller)
more_tokens_tx = Payment(
    account=issuer.classic_address,
    destination=seller.classic_address,
    amount=IssuedCurrencyAmount(
        currency=CURRENCY_CODE,
        issuer=issuer.classic_address,
        value=MORE_TOKENS
    )
)

# Submit
tx_response = submit_and_wait(more_tokens_tx, client, issuer)

# Check Result
result = tx_response.result.get("meta", {}).get("TransactionResult")

if result == "tesSUCCESS":
    print(f"✅ SUCCESS: Issued {MORE_TOKENS} {CURRENCY_CODE} to Seller.")
    print(f"Seller's Address: {seller.classic_address}")
else:
    print(f"❌ Failed: {result}")

How much are the school fee's for the upcoming semester in USD: 500
--- ISSUING 11 MORE PYT ---
✅ SUCCESS: Issued 11 PYT to Seller.
Seller's Address: rQN2DFJP6esSWRtzhZHBKe1DYqp84Nr95h


# Calculate payout per holder of token for each year

In [9]:
import nest_asyncio
nest_asyncio.apply()

from decimal import Decimal
from xrpl.clients import JsonRpcClient
from xrpl.wallet import Wallet
from xrpl.models.requests import AccountLines
from xrpl.models.transactions import Payment
from xrpl.transaction import submit_and_wait
from xrpl.utils import xrp_to_drops

# --- CONFIGURATION ---
client = JsonRpcClient("https://s.altnet.rippletest.net:51234/")

# 1. The "Bank" Wallet (Issuer) that holds the Income
# (In this example, we reuse the issuer from before)
# YOU MUST RE-DEFINE OR LOAD YOUR WALLET HERE if running in a new session
# issuer = Wallet.from_seed("YOUR_SEED_HERE")


# 1. GET INPUT (And convert to float immediately)
income_input = input("How much income did you make for the year in terms of USD: ")

total_income_in_xrp = float(income_input) / latest_xrp_price  # <--- FIX: Convert string "2000" to number 2000.0

# 2. CALCULATE PAYOUT
# Scenario: Each 1 Token gets 0.01% of the income.
# If Income is 2000, 0.01% is 0.2 XRP.
percentage_share = 0.01 / 100  # 0.0001
payout_per_token_float = total_income_in_xrp * percentage_share

# Convert to Decimal for XRP math (XRP libraries prefer Decimal)
PAYOUT_PER_TOKEN = Decimal(str(payout_per_token_float))
print(f"--- STARTING DIVIDEND RUN ---")
print(f"Paying {PAYOUT_PER_TOKEN} XRP per {CURRENCY_CODE} held.")

# 2. SNAPSHOT: Get all holders
# We ask the ledger: "Who has a trust line for PYT with us?"
req = AccountLines(
    account=issuer.classic_address,
    ledger_index="validated"
)
response = client.request(req)
lines = response.result["lines"]

print(f"Found {len(lines)} holders (including empty lines).")

# 3. DISTRIBUTE: Loop through every holder
for line in lines:
    currency = line["currency"]

    # The 'balance' on the trust line is negative from the Issuer's perspective
    # (Because the Issuer "owes" the tokens).
    # So a balance of "-125" means the User holds 125.
    balance = Decimal(line["balance"])

    if currency == CURRENCY_CODE and balance < 0:
        user_holding = abs(balance)
        user_address = line["account"]

        # Calculate Dividend
        payout_amount = user_holding * PAYOUT_PER_TOKEN

        if payout_amount > 0:
            print(f"\nPaying User: {user_address}")
            print(f" - Holds: {user_holding} {CURRENCY_CODE}")
            print(f" - Owed:  {payout_amount} XRP")

            # Send the XRP Payment
            payment_tx = Payment(
                account=issuer.classic_address,
                destination=user_address,
                amount=xrp_to_drops(payout_amount)
            )

            # Submit (In production, you'd batch this or use a queue)
            # We use a try/except block so one failure doesn't stop the whole script
            try:
                pay_response = submit_and_wait(payment_tx, client, issuer)
                res_code = pay_response.result["meta"]["TransactionResult"]
                print(f" - Status: {res_code}")
            except Exception as e:
                print(f" - FAILED: {e}")

print("\n--- DIVIDEND RUN COMPLETE ---")

How much income did you make for the year in terms of USD: 5000
--- STARTING DIVIDEND RUN ---
Paying 0.2304147465437788 XRP per PYT held.
Found 2 holders (including empty lines).

Paying User: rQN2DFJP6esSWRtzhZHBKe1DYqp84Nr95h
 - Holds: 172 PYT
 - Owed:  39.6313364055299536 XRP
 - Status: tesSUCCESS

Paying User: rDx6Mt224DRMCKy2VQ9wTJSzt9rByZLp3o
 - Holds: 3 PYT
 - Owed:  0.6912442396313364 XRP
 - Status: tesSUCCESS

--- DIVIDEND RUN COMPLETE ---


# Gradio Interface

In [27]:
import gradio as gr
import nest_asyncio
import requests
import math
import time
from decimal import Decimal

# XRPL Imports
from xrpl.clients import JsonRpcClient
from xrpl.wallet import generate_faucet_wallet
from xrpl.models.transactions import TrustSet, Payment, OfferCreate
from xrpl.models.amounts import IssuedCurrencyAmount
from xrpl.models.requests import AccountLines
from xrpl.transaction import submit_and_wait
from xrpl.utils import xrp_to_drops

nest_asyncio.apply()

# --- HELPER: FETCH REAL XRP PRICE ---
def get_xrp_price_coingecko():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=ripple&vs_currencies=usd"
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        data = response.json()
        return float(data['ripple']['usd'])
    except Exception as e:
        return None

# --- GLOBAL STATE ---
state = {
    "client": JsonRpcClient("https://s.altnet.rippletest.net:51234/"),
    "issuer": None,
    "seller": None,
    "buyer": None,
    "token_code": "PYT",
    "token_supply": "125",
    "last_trade_tx": None,
    "latest_xrp_price": None
}

# --- STEP 1: SETUP ---
def setup_and_issue(school_fees_usd):
    log = "⏳ Initializing Simulation...\n"
    yield log

    try:
        xrp_price = get_xrp_price_coingecko() or 0.50
        state["latest_xrp_price"] = xrp_price
        log += f"✅ Current XRP Price: ${xrp_price}\n"
        yield log

        xrps_needed = math.ceil(float(school_fees_usd) / xrp_price)
        price_per_token = xrps_needed / 125
        log += f"💰 Funding Needed: {xrps_needed} XRP\n"
        log += f"🏷️ Calculated Initial Price: {price_per_token:.4f} XRP per Token\n\n"
        yield log

        log += "⏳ Generating Wallets (approx 10s)...\n"
        yield log
        state["issuer"] = generate_faucet_wallet(state["client"])
        state["seller"] = generate_faucet_wallet(state["client"])
        state["buyer"] = generate_faucet_wallet(state["client"])

        log += f"✅ Wallets Ready:\nIssuer: {state['issuer'].classic_address}\nSeller: {state['seller'].classic_address}\nBuyer: {state['buyer'].classic_address}\n\n"
        yield log

        log += "⏳ Establishing Trust Line...\n"
        yield log
        trust_tx = TrustSet(
            account=state["seller"].classic_address,
            limit_amount=IssuedCurrencyAmount(
                currency=state["token_code"],
                issuer=state["issuer"].classic_address,
                value="1000000000"
            )
        )
        submit_and_wait(trust_tx, state["client"], state["seller"])

        log += "⏳ Minting Tokens...\n"
        yield log
        payment_tx = Payment(
            account=state["issuer"].classic_address,
            destination=state["seller"].classic_address,
            amount=IssuedCurrencyAmount(
                currency=state["token_code"],
                issuer=state["issuer"].classic_address,
                value=state["token_supply"]
            )
        )
        submit_and_wait(payment_tx, state["client"], state["issuer"])

        log += f"✅ SUCCESS: Issued {state['token_supply']} {state['token_code']} to Seller."
        yield log

    except Exception as e:
        log += f"❌ Error: {str(e)}"
        yield log

# --- STEP 2: TRADE (AGGRESSIVE MATCHING) ---
def execute_trade(trade_mode, qty, price_per_token_xrp):
    state["last_trade_tx"] = None
    if not state["seller"]: return "❌ Error: Run Step 1 first."

    log = ""
    try:
        total_price = float(qty) * float(price_per_token_xrp)
        log += f"📝 Order: {qty} tokens @ {price_per_token_xrp} XRP = {total_price} Total XRP\n"
        log += f"🔄 Mode: {trade_mode}\n\n"

        current_tx = None

        if trade_mode == "Buyer Wants to Buy (Buyer Posts Order)":
            # 1. Buyer Sets Trust
            log += "1️⃣ Buyer setting trust line...\n"
            submit_and_wait(TrustSet(
                account=state["buyer"].classic_address,
                limit_amount=IssuedCurrencyAmount(currency=state["token_code"], issuer=state["issuer"].classic_address, value="1000000000")
            ), state["client"], state["buyer"])

            # 2. Buyer Posts Bid (MAKER)
            log += "2️⃣ Buyer posting BUY offer...\n"
            submit_and_wait(OfferCreate(
                account=state["buyer"].classic_address,
                taker_gets=xrp_to_drops(Decimal(str(total_price))), # Giving XRP
                taker_pays={"currency": state["token_code"], "issuer": state["issuer"].classic_address, "value": str(qty)}
            ), state["client"], state["buyer"])

            log += "   (Waiting 5s for ledger propagation...)\n"
            time.sleep(5)

            # 3. Seller Fills (TAKER) - AGGRESSIVE
            # Seller asks for slightly LESS XRP to guarantee a fill against the Buyer's price
            aggressive_price = total_price * 0.99
            log += f"3️⃣ Seller filling order (Asking {aggressive_price:.2f} XRP to ensure match)...\n"

            current_tx = submit_and_wait(OfferCreate(
                account=state["seller"].classic_address,
                taker_gets={"currency": state["token_code"], "issuer": state["issuer"].classic_address, "value": str(qty)},
                taker_pays=xrp_to_drops(Decimal(str(aggressive_price)))
            ), state["client"], state["seller"])

        else:
            # 1. Seller Posts Ask (MAKER)
            log += "1️⃣ Seller posting SELL offer...\n"
            submit_and_wait(OfferCreate(
                account=state["seller"].classic_address,
                taker_gets={"currency": state["token_code"], "issuer": state["issuer"].classic_address, "value": str(qty)},
                taker_pays=xrp_to_drops(Decimal(str(total_price)))
            ), state["client"], state["seller"])

            log += "   (Waiting 5s for ledger propagation...)\n"
            time.sleep(5)

            # 2. Buyer Sets Trust
            log += "2️⃣ Buyer setting trust line...\n"
            submit_and_wait(TrustSet(
                account=state["buyer"].classic_address,
                limit_amount=IssuedCurrencyAmount(currency=state["token_code"], issuer=state["issuer"].classic_address, value="1000000000")
            ), state["client"], state["buyer"])

            # 3. Buyer Fills (TAKER) - AGGRESSIVE
            # Buyer offers slightly MORE XRP to guarantee a fill
            aggressive_price = total_price * 1.01
            log += f"3️⃣ Buyer purchasing (Offering {aggressive_price:.2f} XRP to ensure match)...\n"

            current_tx = submit_and_wait(OfferCreate(
                account=state["buyer"].classic_address,
                taker_gets=xrp_to_drops(Decimal(str(aggressive_price))),
                taker_pays={"currency": state["token_code"], "issuer": state["issuer"].classic_address, "value": str(qty)}
            ), state["client"], state["buyer"])

        # Update State
        state["last_trade_tx"] = current_tx
        result = current_tx.result.get("meta", {}).get("TransactionResult")
        log += f"\n✅ Trade Executed!\nStatus: {result}\nHash: {current_tx.result.get('hash')}\n"

        if result != "tesSUCCESS":
            log += "⚠️ WARNING: Trade did not succeed. Step 3 will fail.\n"

        return log

    except Exception as e:
        return f"❌ Error: {str(e)}"

# --- STEP 3: ANALYZE (DEBUGGABLE) ---
def analyze_and_mint(next_semester_fees_usd):
    if not state["last_trade_tx"]:
        return "❌ Error: No trade found. Please run Step 2."

    log = "🔍 Analyzing Latest Ledger Entry...\n"

    try:
        current_xrp_price = get_xrp_price_coingecko() or state["latest_xrp_price"] or 0.50
        log += f"💲 Live XRP Price: ${current_xrp_price}\n"

        tx = state["last_trade_tx"]
        tx_hash = tx.result.get('hash')
        log += f"📄 Analyzing Tx: {tx_hash}\n"

        meta = tx.result.get("meta", {})
        affected = meta.get("AffectedNodes", [])
        buyer_addr = state["buyer"].classic_address
        tx_signer = tx.result.get("Account")

        xrp_spent = Decimal(0)
        tokens_rec = Decimal(0)
        fee_drops = Decimal(tx.result.get("Fee", "12"))

        found_buyer = False

        for node in affected:
            entry = node.get("ModifiedNode") or node.get("CreatedNode")
            if not entry: continue

            # Check Buyer's XRP
            if entry["LedgerEntryType"] == "AccountRoot" and entry.get("FinalFields", {}).get("Account") == buyer_addr:
                found_buyer = True
                prev = Decimal(entry.get("PreviousFields", {}).get("Balance", entry["FinalFields"]["Balance"]))
                curr = Decimal(entry["FinalFields"]["Balance"])
                diff = prev - curr

                if tx_signer == buyer_addr:
                    xrp_spent = (diff - fee_drops) / Decimal("1000000")
                else:
                    xrp_spent = diff / Decimal("1000000")

            # Check Buyer's Tokens
            if entry["LedgerEntryType"] == "RippleState":
                bal_data = entry["FinalFields"].get("Balance", {})
                if bal_data.get("currency") == state["token_code"]:
                    prev_val = Decimal(entry.get("PreviousFields", {}).get("Balance", {}).get("value", 0))
                    curr_val = Decimal(bal_data.get("value", 0))
                    tokens_rec = abs(curr_val - prev_val)

        if tokens_rec > 0:
            implied_price = float(xrp_spent / tokens_rec)
            log += f"✅ Analysis Success:\n- Tokens Moved: {tokens_rec}\n- XRP Cost: {xrp_spent}\n- Trade Price: {implied_price:.4f} XRP\n\n"

            fees_in_xrp = float(next_semester_fees_usd) / current_xrp_price
            new_mint = math.ceil(fees_in_xrp / implied_price)

            log += f"🏫 Next Fees: ${next_semester_fees_usd} (~{fees_in_xrp:.2f} XRP)\n"
            log += f"🏭 Minting: {fees_in_xrp:.2f} / {implied_price:.4f} = {new_mint} Tokens\n"

            submit_and_wait(Payment(
                account=state["issuer"].classic_address,
                destination=state["seller"].classic_address,
                amount=IssuedCurrencyAmount(currency=state["token_code"], issuer=state["issuer"].classic_address, value=str(new_mint))
            ), state["client"], state["issuer"])
            log += f"✅ MINTED {new_mint} TOKENS."
        else:
            log += "⚠️ No tokens moved.\n"
            if not found_buyer:
                log += "   (Buyer account was not modified in this transaction)\n"
            log += f"   (Transaction Result: {meta.get('TransactionResult')})\n"
            log += "   Try running the trade again."

        return log
    except Exception as e:
        return f"❌ Error: {str(e)}"

# --- STEP 4: DIVIDENDS ---
def pay_dividends(income_usd):
    if not state["issuer"]: return "❌ Error: Setup first."
    log = "💰 Dividend Run...\n"
    try:
        price = get_xrp_price_coingecko() or 0.50
        income_xrp = float(income_usd) / price
        payout = Decimal(str(income_xrp * 0.0001)) # 0.01%

        log += f"Income: ${income_usd} (~{income_xrp:.2f} XRP)\nPayout: {payout:.6f} XRP/token\n"

        lines = state["client"].request(AccountLines(account=state["issuer"].classic_address, ledger_index="validated")).result["lines"]
        count = 0
        for line in lines:
            if line["currency"] == state["token_code"] and Decimal(line["balance"]) < 0:
                amt = abs(Decimal(line["balance"])) * payout
                if amt > 0:
                    try:
                        submit_and_wait(Payment(
                            account=state["issuer"].classic_address,
                            destination=line["account"],
                            amount=xrp_to_drops(amt)
                        ), state["client"], state["issuer"])
                        log += f"💸 Paid {line['account']}: {amt:.6f} XRP\n"
                        count += 1
                    except: pass
        log += f"✅ Complete. Paid {count} holders."
        return log
    except Exception as e: return f"Error: {e}"

# --- UI ---
with gr.Blocks(title="Student Token Econ", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎓 XRPL Student Token Simulator")

    with gr.Tab("1️⃣ Setup"):
        fees = gr.Number(label="School Fees (USD)", value=2780)
        btn1 = gr.Button("🚀 Setup & Issue", variant="primary")
        log1 = gr.Textbox(label="Log", lines=10)
        btn1.click(setup_and_issue, inputs=[fees], outputs=log1)

    with gr.Tab("2️⃣ Trade"):
        mode = gr.Radio(["Buyer Wants to Buy (Buyer Posts Order)", "Seller Wants to Sell (Seller Posts Order)"], value="Buyer Wants to Buy (Buyer Posts Order)", label="Who Starts?")
        qty = gr.Number(label="Tokens", value=5)
        price = gr.Number(label="Price (XRP) per token", value=12)
        btn2 = gr.Button("💸 Execute Trade")
        log2 = gr.Textbox(label="Log", lines=10)
        btn2.click(execute_trade, inputs=[mode, qty, price], outputs=log2)

    with gr.Tab("3️⃣ Expansion"):
        fees_next = gr.Number(label="Next Fees (USD)", value=3000)
        btn3 = gr.Button("🏭 Analyze & Mint")
        log3 = gr.Textbox(label="Log", lines=10)
        btn3.click(analyze_and_mint, inputs=[fees_next], outputs=log3)

    with gr.Tab("4️⃣ Dividends"):
        inc = gr.Number(label="Income (USD)", value=500)
        btn4 = gr.Button("💰 Pay Dividends")
        log4 = gr.Textbox(label="Log", lines=10)
        btn4.click(pay_dividends, inputs=[inc], outputs=log4)

demo.launch(share=True)

/tmp/ipython-input-1750319681.py:296: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Student Token Econ", theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e8fc6d0cf49491169.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
